In [10]:
%matplotlib widget

import os
import sys

sys.path.insert(0, os.getenv("HOME") + "/pycode/MscThesis/")
import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_plate_number,
    get_postion_number,
)

import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import (
    run_parallel,
    find_state,
    directory_scratch,
    directory_project,
)

In [11]:
plate_number = 721
plate = get_postion_number(plate_number)
# plate = 2
print(plate)
directory = directory_scratch
directory = directory_project
listdir = os.listdir(directory)
list_dir_interest = [
    name
    for name in listdir
    if name.split("_")[-1] == f'Plate{0 if plate<10 else ""}{plate}'
]
dates_datetime = get_dates_datetime(directory, plate)

38


'20210819_0722_Plate38'

In [17]:
i = 139
dirname = get_dirname(dates_datetime[i], plate)

In [29]:
import shutil

j = 0
shutil.copy2("stitching_loop.ijm", f"stitching_loop{j}.ijm")

'stitching_loop0.ijm'

In [43]:
def make_stitching_loop(directory, dirname, index):
    a_file = open(
        f"{path_code}MscThesis/amftrack/pipeline/scripts/stitching_loops/stitching_loop.ijm",
        "r",
    )

    list_of_lines = a_file.readlines()

    list_of_lines[4] = f"mainDirectory = \u0022{directory}\u0022 ;\n"
    list_of_lines[29] = f"\t if(startsWith(list[i],\u0022{dirname}\u0022)) \u007b\n"
    file_name = f"{path_code}MscThesis/amftrack/pipeline/scripts/stitching_loops/stitching_loop{index}.ijm"
    a_file = open(file_name, "w")

    a_file.writelines(list_of_lines)

    a_file.close()

In [54]:
path_code = "/home/cbisot/pycode/"
path_job = "/home/cbisot/bash/job.sh"
from subprocess import call


def run_parallel_stitch(plate, directory, begin, end, num_parallel, time):
    begin_skel = begin // num_parallel
    end_skel = (end) // num_parallel + 1
    listdir = os.listdir(directory)
    list_dir_interest = [
        name
        for name in listdir
        if name.split("_")[-1] == f'Plate{0 if plate<10 else ""}{plate}'
    ]
    dates_datetime = get_dates_datetime(directory, plate)
    for j in range(begin_skel, end_skel):
        start = num_parallel * j + begin % num_parallel
        stop = num_parallel * j + num_parallel + begin % num_parallel
        for k in range(start, stop):
            make_stitching_loop(
                directory, get_dirname(dates_datetime[k], plate), k - start
            )
        ide = int(datetime.now().timestamp())
        my_file = open(path_job, "w")
        my_file.write(
            f"#!/bin/bash \n#Set job requirements \n#SBATCH -N 1 \n#SBATCH -t {time}\n#SBATCH -p normal\n"
        )
        my_file.write(
            f'#SBATCH -o "{path_code}MscThesis/slurm/stitching__{start}_{stop}_{ide}.out" \n'
        )
        for k in range(start, stop):
            my_file.write(
                f"~/Fiji.app/ImageJ-linux64 --headless -macro  {path_code}MscThesis/amftrack/pipeline/scripts/stitching_loops/stitching_loop{k}.ijm &\n"
            )
        my_file.write("wait\n")
        my_file.close()
        call(f"sbatch {path_job}", shell=True)

In [55]:
run_parallel_stitch(13, directory_project, 0, 0, 1, 1)

In [44]:
make_stitching_loop(directory_project, "20200701_1557_Plate13", 0)

In [28]:
directory = directory_project

a_file = open("stitching_loop.ijm", "r")

list_of_lines = a_file.readlines()

list_of_lines[4] = f"mainDirectory = \u0022{directory}\u0022 ;\n"
list_of_lines[29] = f"\t if(startsWith(list[i],\u0022{dirname}\u0022)) \u007b\n"
a_file = open("stitching_loop.ijm", "w")

a_file.writelines(list_of_lines)

a_file.close()